# Тонке налаштування моделей Open AI

Цей ноутбук базується на актуальних рекомендаціях із [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) від Open AI.

Тонке налаштування покращує роботу базових моделей для вашого застосування шляхом додаткового навчання на даних і контексті, що стосуються конкретного випадку використання. Зверніть увагу, що такі техніки, як _few shot learning_ та _retrieval augmented generation_, дозволяють підсилити стандартний запит релевантними даними для підвищення якості. Проте ці підходи обмежені максимальною кількістю токенів, яку підтримує обрана базова модель.

Завдяки тонкому налаштуванню ми фактично перенавчаємо саму модель на потрібних даних (що дозволяє використати набагато більше прикладів, ніж вміщується у вікно токенів) — і розгортаємо _кастомізовану_ версію моделі, якій більше не потрібно надавати приклади під час виконання запиту. Це не лише підвищує ефективність нашого дизайну запитів (ми отримуємо більше гнучкості у використанні вікна токенів для інших цілей), а й потенційно знижує наші витрати (зменшуючи кількість токенів, які потрібно надсилати моделі під час виконання запиту).

Тонке налаштування складається з 4 етапів:
1. Підготувати тренувальні дані та завантажити їх.
1. Запустити навчання, щоб отримати модель із тонким налаштуванням.
1. Оцінити модель із тонким налаштуванням і повторити для покращення якості.
1. Розгорнути модель із тонким налаштуванням для використання, коли результат вас влаштовує.

Зверніть увагу, що не всі базові моделі підтримують тонке налаштування — [перевіряйте документацію OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) для отримання найсвіжішої інформації. Також можна виконати тонке налаштування вже налаштованої моделі. У цьому уроці ми будемо використовувати `gpt-35-turbo` як цільову базову модель для тонкого налаштування.

---


### Крок 1.1: Підготуйте свій датасет

Давайте створимо чат-бота, який допоможе вам розібратися в періодичній таблиці елементів, відповідаючи на запитання про елемент у формі лимерика. У _цьому_ простому уроці ми просто створимо датасет для навчання моделі з кількома прикладами відповідей, які демонструють очікуваний формат даних. У реальному випадку використання вам потрібно буде створити датасет із набагато більшою кількістю прикладів. Також ви можете скористатися відкритим датасетом (для вашої предметної області), якщо такий існує, і відформатувати його для використання у донавчанні.

Оскільки ми зосереджуємось на `gpt-35-turbo` і очікуємо одноразову відповідь (chat completion), ми можемо створити приклади, використовуючи [цей рекомендований формат](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), який відповідає вимогам OpenAI для завершення чату. Якщо ви очікуєте багатокрокову розмову, слід використовувати [формат для багатокрокових прикладів](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), який містить параметр `weight`, щоб вказати, які повідомлення слід використовувати (або ні) під час донавчання.

У цьому уроці ми використаємо простіший формат для одноразових відповідей. Дані зберігаються у [jsonl-форматі](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) — 1 запис на рядок, кожен представлений як об’єкт у форматі JSON. Нижче наведено фрагмент із 2 записів як приклад — повний набір прикладів (10 штук), який ми використаємо для нашого уроку з донавчання, дивіться у [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl). **Зверніть увагу:** Кожен запис _має_ бути визначений в одному рядку (не розбитий на кілька рядків, як це зазвичай буває у форматованому JSON-файлі)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

У реальному випадку використання вам знадобиться набагато більший набір прикладів для отримання якісних результатів — тут потрібно знайти баланс між якістю відповідей і часом/витратами на донавчання. Ми використовуємо невеликий набір, щоб швидко пройти процес і показати, як це працює. Дивіться [цей приклад з OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) для більш складного уроку з донавчання.


### Крок 1.2 Завантаження вашого датасету

Завантажте дані за допомогою Files API [як описано тут](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Зверніть увагу, що для запуску цього коду спочатку потрібно виконати такі дії:
 - Встановити Python-пакет `openai` (переконайтеся, що використовуєте версію >=0.28.0 для нових можливостей)
 - Встановити змінну середовища `OPENAI_API_KEY` зі своїм OpenAI API ключем
Щоб дізнатися більше, перегляньте [інструкцію з налаштування](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), яка надається для цього курсу.

Тепер запустіть код, щоб створити файл для завантаження зі свого локального JSONL-файлу.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Крок 2.1: Створіть завдання для донавчання за допомогою SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Крок 2.2: Перевірте статус завдання

Ось кілька дій, які можна виконати за допомогою API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` – Показати останні n завдань тонкого налаштування
- `client.fine_tuning.jobs.retrieve(<job_id>)` – Отримати деталі конкретного завдання тонкого налаштування
- `client.fine_tuning.jobs.cancel(<job_id>)` – Скасувати завдання тонкого налаштування
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` – Показати до n подій із завдання
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Перший крок цього процесу — _перевірка навчального файлу_, щоб переконатися, що дані мають правильний формат.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Крок 2.3: Відстежуйте події для моніторингу прогресу


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Крок 2.4: Перегляньте статус у панелі керування OpenAI


Ви також можете переглянути статус, відвідавши сайт OpenAI та відкривши розділ _Fine-tuning_ на платформі. Тут ви побачите статус поточного завдання, а також зможете відстежувати історію попередніх запусків. На цьому скріншоті видно, що попередній запуск завершився невдало, а другий — успішно. Для розуміння: це сталося тому, що під час першого запуску був використаний JSON-файл з неправильно відформатованими записами — після виправлення другий запуск завершився успішно, і модель стала доступною для використання.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.uk.png)


Ви також можете переглядати статусні повідомлення та метрики, прокручуючи далі вниз у візуальній панелі приладів, як показано нижче:

| Повідомлення | Метрики |
|:---|:---|
| ![Повідомлення](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.uk.png) |  ![Метрики](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.uk.png)|


### Крок 3.1: Отримайте ID і протестуйте донавчану модель у коді


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Крок 3.2: Завантаження та тестування донавченого моделі у Playground

Тепер ви можете протестувати донавчену модель двома способами. По-перше, ви можете відвідати Playground і скористатися випадаючим списком Models, щоб обрати вашу нову донавчену модель із запропонованих варіантів. Інший спосіб — скористатися опцією "Playground", яка відображається у панелі Fine-tuning (див. скріншот вище). Вона відкриває _порівняльний_ режим, у якому версії базової та донавченої моделі показані поруч для швидкої оцінки.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.uk.png)

Просто заповніть системний контекст, який ви використовували у своїх тренувальних даних, і введіть тестове питання. Ви помітите, що обидві сторони оновлюються з однаковим контекстом і питанням. Запустіть порівняння, і ви побачите різницю у відповідях між ними. _Зверніть увагу, як донавчена модель формує відповідь у тому форматі, який ви задали у своїх прикладах, тоді як базова модель просто слідує системному запиту_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.uk.png)

Ви також помітите, що під час порівняння відображається кількість токенів для кожної моделі та час, витрачений на інференцію. **Цей конкретний приклад є спрощеним і призначений лише для демонстрації процесу, але не відображає реальний набір даних чи сценарій**. Можливо, ви помітите, що в обох прикладах однакова кількість токенів (системний контекст і запит користувача ідентичні), але донавчена модель витрачає більше часу на інференцію (кастомна модель).

У реальних сценаріях ви не будете використовувати такий простий приклад, а будете донавчати модель на реальних даних (наприклад, каталог товарів для служби підтримки клієнтів), де якість відповіді буде набагато помітнішою. У _такому_ випадку, щоб отримати таку ж якість відповіді від базової моделі, доведеться більше працювати над інженерією підказок, що збільшить використання токенів і, можливо, час обробки для інференції. _Щоб спробувати це на практиці, перегляньте приклади донавчання в OpenAI Cookbook._



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичний переклад може містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для отримання критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильне тлумачення, що виникли внаслідок використання цього перекладу.
